In [2]:
import pandas as pd

# 讀取資料
file_path = 'course_data05.csv'
data = pd.read_csv(file_path)

# 清除教師欄位為 NaN 的資料
data = data[data['教師姓名*:主負責老師'].notna()]

# 檢查資料的基本資訊
print("資料形狀:", data.shape)
print("\n欄位名稱:")
print(data.columns.tolist())

print("\n資料類型:")
print(data.dtypes)

# 簡要預覽資料
print("\n前五行數據:")
print(data.head())

print("\n資料中的缺失值檢查:")
missing_values = data.isnull().sum()
print(missing_values[missing_values > 0])

# 統計摘要
print("\n數值型資料的統計摘要:")
print(data.describe())

# 字串型資料的基礎分析
print("\n字串型欄位的值計數:")
for col in data.select_dtypes(include=['object']).columns:
    print(f"\n欄位: {col}")
    print(data[col].value_counts().head())


資料形狀: (1183, 12)

欄位名稱:
['學年期', '系所名稱', '系號-序號課程碼-分班碼屬性碼', '年級班別組別', '類別', '科目名稱(連結課程地圖)             備註 \n             限選條件', '學分選必修', '教師姓名*:主負責老師', '已選課人數/餘額', '時間/教室', '課綱/Moodle', 'Syllabus Link']

資料類型:
學年期                                                object
系所名稱                                               object
系號-序號課程碼-分班碼屬性碼                                    object
年級班別組別                                             object
類別                                                 object
科目名稱(連結課程地圖)             備註 \n             限選條件    object
學分選必修                                              object
教師姓名*:主負責老師                                        object
已選課人數/餘額                                           object
時間/教室                                              object
課綱/Moodle                                          object
Syllabus Link                                      object
dtype: object

前五行數據:
      學年期      系所名稱                系號-序號課程碼-分班碼屬性碼 年級班別組別  類別  \
2  0108-2

In [5]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# 讀取資料
file_path = 'course_data05.csv'
data = pd.read_csv(file_path)

# 清除教師欄位為 NaN 的資料
data = data[data['教師姓名*:主負責老師'].notna()]

# 清除系所名稱或科目名稱欄位為 NaN 的資料
data = data[data['系所名稱'].notna() & data['科目名稱(連結課程地圖)             備註 \n             限選條件'].notna()]

# 對系所名稱欄位進行 Label Encoding
label_encoder = LabelEncoder()
data['系所名稱_label'] = label_encoder.fit_transform(data['系所名稱'].fillna('Unknown'))

# 清掉不需要的欄位
data = data.drop(['系號-序號課程碼-分班碼屬性碼', '年級班別組別', '課綱/Moodle', 'Syllabus Link'], axis=1)

# 對類別欄位進行 One-Hot Encoding
data = pd.get_dummies(data, columns=['類別'], prefix='類別')

# 檢查處理後的資料
print("資料形狀:", data.shape)
print("\n欄位名稱:")
print(data.columns.tolist())

print("\n前五行數據:")
print(data.head())

資料形狀: (1026, 10)

欄位名稱:
['學年期', '系所名稱', '科目名稱(連結課程地圖)             備註 \n             限選條件', '學分選必修', '教師姓名*:主負責老師', '已選課人數/餘額', '時間/教室', '系所名稱_label', '類別_實習', '類別_講義']

前五行數據:
      學年期      系所名稱 科目名稱(連結課程地圖)             備註 \n             限選條件  學分選必修  \
2  0108-2  數學系 MATH                                         服務學習（二）  0  必修   
3  0108-2  數學系 MATH                                          微積分（二）  4  必修   
5  0108-2  數學系 MATH                                         線性代數（二）  3  必修   
7  0108-2  數學系 MATH                                        普通物理學（二）  3  必修   
8  0108-2  數學系 MATH                                      計算機概論與程式語言  3  必修   

  教師姓名*:主負責老師    已選課人數/餘額                                   時間/教室  系所名稱_label  \
2         林育竹  45/網路選課不開放                                    [1]N           0   
3         林育竹      454/16  [1]2~3 理學教學大樓 36202[3]5~6 理學教學大樓 36202           0   
5         劉珈銘      647/13      [3]2 資訊大樓 格致廳小講堂[5]3~4 資訊大樓 格致廳小講堂           0   
7         游輝樟       54/16      

In [15]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# 讀取資料
file_path = 'course_data05.csv'
data = pd.read_csv(file_path)

# 清除教師欄位為 NaN 的資料
data = data[data['教師姓名*:主負責老師'].notna()]

# 清除系所名稱或科目名稱欄位為 NaN 的資料
data = data[data['系所名稱'].notna() & data['科目名稱(連結課程地圖)             備註 \n             限選條件'].notna()]

# 對系所名稱欄位進行 Label Encoding
label_encoder = LabelEncoder()
data['系所名稱_label'] = label_encoder.fit_transform(data['系所名稱'].fillna('Unknown'))

# 安全分割學分選必修欄位，處理異常數據並打印異常值
def split_credit_field(value):
    parts = value.split(' ', 1)  # 僅分割一次
    if len(parts) == 2:
        return parts[0], parts[1]
    print(f"異常數據: {value}")
    return None, None

# 對學分選必修欄位進行分割，處理異常數據
data['學分'], data['必選修'] = zip(*data['學分選必修'].map(split_credit_field))
data = data.dropna(subset=['學分', '必選修'])  # 移除異常數據

data = data.drop('學分選必修', axis=1)

# 拆分時間/教室欄位為時間和教室兩個特徵
def split_time_classroom(value, row):
    import re
    if value == '未定':
        return None, row['時間/教室'].split(' ', 1)[-1] if ' ' in row['時間/教室'] else None
    if re.search(r'\[\d+\]N', value):
        return None, None
    matches = re.findall(r'\[\d+\][^\[]+', value)
    if len(matches) == 1:  # 僅一個時間段和教室
        try:
            time1, classroom1 = matches[0].split(' ', 1)
            return time1, classroom1
        except ValueError:
            return None, None
    elif len(matches) == 2:  # 兩個時間段和教室
        try:
            time1, classroom1 = matches[0].split(' ', 1)
            time2, classroom2 = matches[1].split(' ', 1)
            if classroom1 == classroom2:
                return f"{time1},{time2}", classroom1
            return f"{time1},{time2}", f"{classroom1};{classroom2}"
        except ValueError:
            return None, None
    return None, None

# 應用拆分函數
data['時間'], data['教室'] = zip(*data.apply(lambda row: split_time_classroom(row['時間/教室'], row), axis=1))
data = data.dropna(subset=['時間', '教室'])  # 移除異常數據

data = data.drop('時間/教室', axis=1)

# 清掉不需要的欄位
data = data.drop(['系號-序號課程碼-分班碼屬性碼', '年級班別組別', '課綱/Moodle', 'Syllabus Link'], axis=1)

# 對類別欄位進行 One-Hot Encoding
data = pd.get_dummies(data, columns=['類別'], prefix='類別')

# 將清理後的資料保存為新的 CSV
output_file_path = 'cleaned_course_data.csv'
data.to_csv(output_file_path, index=False)

print(f"清理後的數據已保存到: {output_file_path}")


清理後的數據已保存到: cleaned_course_data.csv
